In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
data_dir = 'resources'
file_name = 'Task_scoring_modeller_Nov2016.xlsx'

file_path = os.path.join(data_dir, file_name)

In [4]:
df = pd.read_excel(file_path, sheet_name='Data', index_col='client_id')

In [5]:
df.shape

(32144, 38)

In [6]:
df.head(10)

,status,mobile_operator,gender,email_domain,application_hour,application_day,application_weekday,asked_amount,term,application_type,client_age,registration_hour,actual_city,reg_req_date_match,days_since_reg,accept_news,email_contains_dob,bank,monthly_income,monthly_liabilities,work_status,p1m_req_cnt,p6m_req_cnt,loans_taken_p6m,loans_taken_p12m,sum_loans_taken_p6m,sum_loans_taken_p12m,sum_contract_amount,max_contract_amount,avg_contract_amount,sum_ConsumerLoan,days_since_most_recent_cr_start,days_since_first_cr_start,ci_p1m_req_count,ci_p3m_req_count,ci_p6m_req_count,closed_credits_count,open_credits_count
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,GOOD,LaiLai,MALE,hotmail.com,12,9,1,500,30.0,Type1,30,12,თბილისი,1,0,1.0,0,BG,500.0,500.0,Working,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,4.0,4.0,0.0,1.0
2,GOOD,Beeline,MALE,mail.ru,12,7,3,300,30.0,Type1,29,10,ახალქალაქი,0,34,1.0,0,BG,900.0,0.0,Individual entrepreneur,3.0,4,1.0,1.0,300.0,300.0,250.0,250.0,250.0,250.0,23.0,23.0,0.0,3.0,3.0,0.0,1.0
3,GOOD,Bali,FEMALE,gmail.com,20,29,0,200,30.0,Type1,20,20,კასპი,1,0,1.0,0,BG,300.0,100.0,Working,0.0,0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,26.0,26.0,0.0,1.0,2.0,0.0,1.0
4,GOOD,LaiLai,FEMALE,mail.ru,11,11,4,500,30.0,Type1,36,19,რუსთავი,0,42,1.0,0,BG,500.0,0.0,Working,0.0,1,1.0,1.0,300.0,300.0,250.0,250.0,250.0,250.0,26.0,26.0,0.0,2.0,2.0,0.0,1.0
5,GOOD,Beeline,FEMALE,mail.ru,13,8,4,250,30.0,Type1,53,13,ზესტაფონი,0,42,1.0,1,LB,500.0,0.0,Self-employed person,0.0,1,1.0,1.0,300.0,300.0,250.0,250.0,250.0,250.0,28.0,28.0,0.0,2.0,2.0,0.0,1.0
6,GOOD,Magticom,FEMALE,iliauni.edu.ge,14,25,4,250,30.0,Type1,20,14,თბილისი,1,0,1.0,0,BG,300.0,50.0,Working,0.0,0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,29.0,29.0,0.0,1.0,2.0,0.0,1.0
7,GOOD,LaiLai,MALE,gmail.com,12,7,0,500,30.0,Type1,21,12,თბილისი,1,0,1.0,0,TB,900.0,0.0,Working,0.0,0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,29.0,29.0,0.0,1.0,1.0,0.0,1.0
8,INDETERMINED,Bali,MALE,mail.ru,14,3,3,100,30.0,Type1,20,14,თბილისი,1,0,1.0,0,BG,300.0,50.0,Working,0.0,0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,29.0,29.0,0.0,1.0,1.0,0.0,1.0
9,GOOD,Magticom,FEMALE,bk.ru,15,22,1,300,30.0,Type2,23,12,თბილისი,0,29,1.0,1,TB,100.0,350.0,Working,1.0,1,1.0,1.0,200.0,200.0,200.0,200.0,200.0,200.0,29.0,29.0,0.0,1.0,1.0,0.0,1.0


In [7]:
df['status'].unique()

array(['GOOD', 'INDETERMINED', 'BAD'], dtype=object)

In [8]:
df = df[df['status'] != 'INDETERMINED']

In [9]:
df['status'] = df.apply(lambda row: 1 if row['status'] == 'GOOD' else 0, axis=1)

In [10]:
df.shape

(31439, 38)

In [11]:
def calc_summary(df):
    return pd.DataFrame(
        data=np.array([df.dtypes, df.nunique(), df.isna().sum()]).T, 
        columns=['dtype', 'nunique', 'nan_count'], 
        index=df.columns
    )

In [12]:
calc_summary(df)

,dtype,nunique,nan_count
status,int64,2,0
mobile_operator,object,10,1
gender,object,2,0
email_domain,object,499,0
application_hour,int64,24,0
application_day,int64,31,0
application_weekday,int64,7,0
asked_amount,int64,23,0
term,float64,26,1
application_type,object,4,0


In [13]:
df.dropna(inplace=True)

In [14]:
df.shape

(29915, 38)

In [15]:
def make_abs_freq_dict(items):
    return {x: items.count(x) for x in set(items)}


def make_rel_freq_dict(items):
    abs_freq_dict = make_abs_freq_dict(items)
    values_total = sum(abs_freq_dict.values())
    return {item: abs_freq / values_total for item, abs_freq in abs_freq_dict.items()}

In [16]:
def list_rare_values(items, threshold=0.01):
    rel_freq_dict = make_rel_freq_dict(items)
    return [item for item, rel_freq in rel_freq_dict.items() if rel_freq < threshold]

In [17]:
rare_mobile_operators = set(list_rare_values(df['mobile_operator'].tolist()))
rare_mobile_operators

{'Silknet', 'Unknown operator', 'Wrong length', 'Wrong mobile'}

In [18]:
rare_email_domains = set(list_rare_values(df['email_domain'].tolist()))
len(rare_email_domains)

477

In [19]:
rare_actual_cities = set(list_rare_values(df['actual_city'].tolist()))
len(rare_actual_cities)

71

In [20]:
rare_banks = set(list_rare_values(df['bank'].tolist()))
rare_banks

{'BS', 'BT', 'CR', 'CS', 'FG', 'HB', 'IN', 'KS', 'PC', 'PG', 'VT', 'ZB'}

In [21]:
make_abs_freq_dict(df['work_status'].tolist())

{'Individual entrepreneur': 1513,
 'Pensioner': 137,
 'Self-employed person': 5014,
 'Social beneficiaries': 92,
 'Student': 278,
 'Unemployed': 186,
 'Working': 22693,
 'შპს ფასანაური': 1,
 'შპს ჯეკა გრუპი / გასართობი ცენტრი ბუმბული': 1}

In [22]:
df = df[~df['work_status'].isin({'შპს ფასანაური', 'შპს ჯეკა გრუპი / გასართობი ცენტრი ბუმბული'})]

In [23]:
def replace_rare_values(value, rare_values, default_value='RARE'):
    return default_value if value in rare_values else value


def replace_rare_mobile_operators(value, default_value='RARE'):
    return replace_rare_values(value, rare_mobile_operators, default_value=default_value)


def replace_rare_email_domains(value, default_value='RARE'):
    return replace_rare_values(value, rare_email_domains, default_value=default_value)


def replace_rare_actual_cities(value, default_value='RARE'):
    return replace_rare_values(value, rare_actual_cities, default_value=default_value)


def replace_rare_banks(value, default_value='RARE'):
    return replace_rare_values(value, rare_banks, default_value=default_value)

In [24]:
df['mobile_operator'] = df.apply(lambda row: replace_rare_mobile_operators(row['mobile_operator']), axis=1)
df['email_domain'] = df.apply(lambda row: replace_rare_email_domains(row['email_domain']), axis=1)
df['actual_city'] = df.apply(lambda row: replace_rare_actual_cities(row['actual_city']), axis=1)
df['bank'] = df.apply(lambda row: replace_rare_banks(row['bank']), axis=1)

In [25]:
calc_summary(df)

,dtype,nunique,nan_count
status,int64,2,0
mobile_operator,object,7,0
gender,object,2,0
email_domain,object,8,0
application_hour,int64,24,0
application_day,int64,31,0
application_weekday,int64,7,0
asked_amount,int64,22,0
term,float64,26,0
application_type,object,4,0


In [26]:
categorical_columns = [
    'mobile_operator', 'gender', 'email_domain', 'application_weekday', 'application_type', 'registration_hour', 'actual_city',
    'bank', 'work_status'
]

df = pd.get_dummies(
    data=df, prefix_sep='__', 
    columns=categorical_columns,
    drop_first=True
)

In [27]:
df.shape

(29913, 102)

In [28]:
df.head()

,status,application_hour,application_day,asked_amount,term,client_age,reg_req_date_match,days_since_reg,accept_news,email_contains_dob,monthly_income,monthly_liabilities,p1m_req_cnt,p6m_req_cnt,loans_taken_p6m,loans_taken_p12m,sum_loans_taken_p6m,sum_loans_taken_p12m,sum_contract_amount,max_contract_amount,avg_contract_amount,sum_ConsumerLoan,days_since_most_recent_cr_start,days_since_first_cr_start,ci_p1m_req_count,ci_p3m_req_count,ci_p6m_req_count,closed_credits_count,open_credits_count,mobile_operator__Bani,mobile_operator__Beeline,mobile_operator__Geocell,mobile_operator__LaiLai,mobile_operator__Magticom,mobile_operator__RARE,gender__MALE,email_domain__bk.ru,email_domain__gmail.com,email_domain__inbox.ru,email_domain__list.ru,email_domain__mail.ru,email_domain__rambler.ru,email_domain__yahoo.com,application_weekday__1,application_weekday__2,application_weekday__3,application_weekday__4,application_weekday__5,application_weekday__6,application_type__Type2,application_type__Type3,application_type__Type4,registration_hour__1,registration_hour__2,registration_hour__3,registration_hour__4,registration_hour__5,registration_hour__6,registration_hour__7,registration_hour__8,registration_hour__9,registration_hour__10,registration_hour__11,registration_hour__12,registration_hour__13,registration_hour__14,registration_hour__15,registration_hour__16,registration_hour__17,registration_hour__18,registration_hour__19,registration_hour__20,registration_hour__21,registration_hour__22,registration_hour__23,actual_city__ახალციხე,actual_city__ბათუმი,actual_city__გორი,actual_city__გურჯაანი,actual_city__ზესტაფონი,actual_city__ზუგდიდი,actual_city__თბილისი,actual_city__თელავი,actual_city__ოზურგეთი,actual_city__რუსთავი,actual_city__სამტრედია,actual_city__სენაკი,actual_city__სხვა,actual_city__ფოთი,actual_city__ქობულეთი,actual_city__ქუთაისი,actual_city__ხაშური,bank__BR,bank__LB,bank__RARE,bank__TB,work_status__Pensioner,work_status__Self-employed person,work_status__Social beneficiaries,work_status__Student,work_status__Unemployed,work_status__Working
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,12,9,500,30.0,30,1,0,1.0,0,500.0,500.0,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,4.0,4.0,0.0,1.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,12,7,300,30.0,29,0,34,1.0,0,900.0,0.0,3.0,4,1.0,1.0,300.0,300.0,250.0,250.0,250.0,250.0,23.0,23.0,0.0,3.0,3.0,0.0,1.0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,20,29,200,30.0,20,1,0,1.0,0,300.0,100.0,0.0,0,1.0,1.0,100.0,100.0,100.0,100.0,100.0,100.0,26.0,26.0,0.0,1.0,2.0,0.0,1.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,11,11,500,30.0,36,0,42,1.0,0,500.0,0.0,0.0,1,1.0,1.0,300.0,300.0,250.0,250.0,250.0,250.0,26.0,26.0,0.0,2.0,2.0,0.0,1.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,1,13,8,250,30.0,53,0,42,1.0,1,500.0,0.0,0.0,1,1.0,1.0,300.0,300.0,250.0,250.0,250.0,250.0,28.0,28.0,0.0,2.0,2.0,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [29]:
calc_summary(df)

,dtype,nunique,nan_count
status,int64,2,0
application_hour,int64,24,0
application_day,int64,31,0
asked_amount,int64,22,0
term,float64,26,0
client_age,int64,46,0
reg_req_date_match,int64,2,0
days_since_reg,int64,540,0
accept_news,float64,2,0
email_contains_dob,int64,2,0


In [30]:
status_df = pd.DataFrame(data=df['status'], index=df.index)
df.drop(labels=['status'], axis=1, inplace=True)

In [31]:
numeric_columns = df.select_dtypes(include=[np.int64, np.float64]).columns.tolist()

In [32]:
df[numeric_columns] = MinMaxScaler().fit_transform(df[numeric_columns].values)

In [33]:
X = df.values
y = status_df.values

In [34]:
print(X.shape)
print(y.shape)

(29913, 101)
(29913, 1)


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [36]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(26921, 101)
(2992, 101)
(26921, 1)
(2992, 1)


In [37]:
l2_penalty_powers = np.random.uniform(low=-7, high=-2, size=10)
l2_penalties = 10 ** l2_penalty_powers

In [38]:
for i, l2_penalty in enumerate(l2_penalties):
    print('#exp: {}'.format(i + 1))
    print('l2_penalty: {:.7f}'.format(l2_penalty))
    C = 1 / l2_penalty
    log_reg = LogisticRegression(solver='lbfgs', C=C, random_state=1).fit(X_train, y_train.ravel())
    y_pred = log_reg.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print('accuracy: {:.4f}'.format(accuracy))
    print()

#exp: 1
l2_penalty: 0.0006874
accuracy: 0.8890

#exp: 2
l2_penalty: 0.0000057
accuracy: 0.8890

#exp: 3
l2_penalty: 0.0024602
accuracy: 0.8890

#exp: 4
l2_penalty: 0.0000121
accuracy: 0.8890

#exp: 5
l2_penalty: 0.0001559
accuracy: 0.8897

#exp: 6
l2_penalty: 0.0064425
accuracy: 0.8887

#exp: 7
l2_penalty: 0.0006809
accuracy: 0.8890

#exp: 8
l2_penalty: 0.0000002
accuracy: 0.8894

#exp: 9
l2_penalty: 0.0000051
accuracy: 0.8894

#exp: 10
l2_penalty: 0.0009302
accuracy: 0.8890

